In this exercise we will classify stimulus classes using the Haxby et al. data. You should first obtain the data using the command:

wget http://data.pymvpa.org/datasets/haxby2001/subj1-2010.01.14.tar.gz

and set the datadir variable accordingly

In [1]:
import nipype.algorithms.modelgen as model   # model generation
import nipype.interfaces.fsl as fsl          # fsl
from nipype.interfaces.base import Bunch
import os,json,glob
import numpy
import nibabel
import nilearn.plotting
import sklearn.multiclass
from sklearn.svm import LinearSVC
import sklearn.metrics
import sklearn.cross_validation
from nilearn.input_data import NiftiMasker
import scipy.stats
import random

%matplotlib inline
import matplotlib.pyplot as plt

from haxby_data import HaxbyData


datadir='/Users/poldrack/data_unsynced/haxby/subj1'

print 'Using data from',datadir

haxbydata=HaxbyData(datadir)

boldimg=nibabel.load(haxbydata.boldfile)

if not os.path.exists(haxbydata.brainmaskfile):
    bet=fsl.BET()
    bet.inputs.in_file=haxbydata.boldfile
    bet.inputs.out_file=haxbydata.boldfile.replace('.nii.gz','_brain.nii.gz')
    bet.inputs.functional=True
    bet.inputs.mask=True
    bet.run()


brainmaskimg=nibabel.load(haxbydata.brainmaskfile)
vtmaskimg=nibabel.load(haxbydata.vtmaskfile)


Using data from /Users/poldrack/data_unsynced/haxby/subj1


Set up model


In [2]:
modeldir=os.path.join(datadir,'blockmodel')
# no way to specify the output directory, so we just chdir into the 
# desired output directory
if not os.path.exists(modeldir):
    os.mkdir(modeldir)
os.chdir(modeldir)


Estimate the model with a separate condition for each block using FSL.  This will take several hours to finish.

In [3]:
contrasts=[]

for i in range(len(haxbydata.conditions)):
    contrasts.append([haxbydata.conditions[i],'T',[haxbydata.conditions[i]],[1]])


# this is how one could do it using FSL - this is VERY slow, so let's compute the GLM on our own
if not os.path.exists(os.path.join(modeldir,'stats')):
    
    
    info = [Bunch(conditions=haxbydata.conditions,
                  onsets=haxbydata.onsets,
                  durations=haxbydata.durations)
           ]
    print 'SpecifyModel'
    s = model.SpecifyModel()
    s.inputs.input_units = 'secs'
    s.inputs.functional_runs = [haxbydata.boldbrainfile]
    s.inputs.time_repetition = haxbydata.tr
    s.inputs.high_pass_filter_cutoff = 128.
    s.inputs.subject_info = info
    s.run()

    print 'level1design'
    level1design = fsl.model.Level1Design()
    level1design.inputs.interscan_interval = haxbydata.tr
    level1design.inputs.bases = {'dgamma':{'derivs': False}}
    level1design.inputs.session_info = s._sessinfo
    level1design.inputs.model_serial_correlations=False
    level1design.inputs.contrasts=contrasts
    level1info=level1design.run() 
    
    fsf_file=os.path.join(modeldir,'run0.fsf')
    matfile=fsf_file.replace(".fsf",".mat")
    event_files=glob.glob(os.path.join(modeldir,'ev*txt'))

    print 'modelgen'
    modelgen=fsl.model.FEATModel()
    modelgen.inputs.fsf_file=fsf_file
    modelgen.inputs.ev_files=event_files
    modelgen.run()

    print 'FILMGLS'
    fgls = fsl.FILMGLS(autocorr_noestimate=True)
    fgls.inputs.in_file =haxbydata.boldbrainfile
    fgls.inputs.design_file = os.path.join(modeldir,'run0.mat')
    fgls.inputs.threshold = 10
    fgls.inputs.results_dir = os.path.join(modeldir,'stats')
    fgls.inputs.tcon_file=os.path.join(modeldir,'run0.con')
    res = fgls.run() 

else:
    print 'stats have already been run - using existing files'


stats have already been run - using existing files


Load the zstat images that we will use as our block-by-block signal estimates

In [5]:
use_whole_brain=False

if not os.path.exists(os.path.join(modeldir,'zstatdata.nii.gz')):
    zstatdata=numpy.zeros((boldimg.shape[0],boldimg.shape[1],boldimg.shape[2],len(haxbydata.conditions)))
    for i in range(len(conditions)):
        zstatdata[:,:,:,i]=nibabel.load(os.path.join(modeldir,'stats/zstat%d.nii.gz'%int(i+1))).get_data()

    zstatimg=nibabel.Nifti1Image(zstatdata,affine=brainmaskimg.get_affine())
    zstatimg.to_filename(os.path.join(modeldir,'zstatdata.nii.gz'))

if use_whole_brain:
    maskimg=haxbydata.brainmaskfile
else:
    maskimg=haxbydata.vtmaskfile
    
nifti_masker = NiftiMasker(mask_img=maskimg, standardize=False)
fmri_masked = nifti_masker.fit_transform(os.path.join(modeldir,'zstatdata.nii.gz'))

# include faces and cats
condition_mask = numpy.logical_or(haxbydata.condnums == 2,
                               haxbydata.condnums == 3)
fmri_masked = fmri_masked[condition_mask,:]
condlabels=haxbydata.condnums[condition_mask]
runlabels=haxbydata.runs[condition_mask]

Now let's do a leave-one-run out classifier.

In [13]:
def shuffle_within_runs(labels,runs):
    for r in numpy.unique(runs):
        l=labels[runs==r]
        random.shuffle(l)
        labels[runs==r]=l
    return labels


def run_classifier(fmri_masked,condlabels,runs,baseclf,shuffle_labels=False):
    cv = sklearn.cross_validation.LeaveOneLabelOut(labels=runs)

    pred=numpy.zeros(len(runs)) # predicted class

    if len(numpy.unique(condlabels))>2:
        clf=sklearn.multiclass.OneVsRestClassifier(baseclf)
    else:
        clf=baseclf
    
    for train,test in cv:
        testdata=fmri_masked[test,:]
        traindata=fmri_masked[train,:]
        trainlabels=condlabels[train]
        if shuffle_labels:
            shuffle_within_runs(trainlabels,runs[train])
        clf.fit(traindata,trainlabels)
        pred[test]=clf.predict(testdata)
        
    confmtx=sklearn.metrics.confusion_matrix(condlabels,pred)
    acc=sklearn.metrics.accuracy_score(condlabels,pred)
    return pred,confmtx,acc

pred,confmtx,acc=run_classifier(fmri_masked,condlabels,runlabels,LinearSVC())



In [14]:
print confmtx
print 'Accuracy score:',acc

[[9 3]
 [4 8]]
Accuracy score: 0.708333333333


In [ ]:
nperms=500
randacc=numpy.zeros(nperms)
for i in range(nperms):
    p,c,randacc[i]=run_classifier(fmri_masked,haxbydata.condlabels,runlabels,LinearSVC(),shuffle_labels=True)


In [ ]:
pct=scipy.stats.percentileofscore(randacc,acc)
print 'Pval:',(100-pct)/100.0


Now let's set up a searchlight analysis

In [ ]:
from nilearn.image import index_img
fmri_img = index_img(zstatimg, condition_mask)


In [ ]:
fmri_img